In [1]:
%loadext autoreload
%autoreload 2

UsageError: Line magic function `%loadext` not found.


In [1]:
import deep_fib
import utils

In [4]:
train, test = utils.data.get_train_test_split()
dataset = utils.data.Marconi100Dataset()

SyntaxError: invalid syntax (<ipython-input-4-3fbd185bc487>, line 1)

In [ ]:
hpcsv = hpc.copy()
hpcsv[inputs] = (hpcsv[inputs] - hpcsv[inputs].mean()) / hpcsv[inputs].std()
nn.plot_dataframe(hpcsv[inputs], figsize=figsize)